导入所需模块

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print(tf.__version__)

获取tensorflow.keras.datasets.mnist数据集

In [12]:
(train_data, train_labels), (test_data, test_labels) = keras.datasets.mnist.load_data()

In [3]:
print(f"shape of train_data: {train_data.shape},type of train_label: {type(train_labels)}")

shape of train_data: (60000, 28, 28),type of train_label: <class 'numpy.ndarray'>


数据预处理

In [13]:
train_data =train_data / np.float32(255)
train_labels = train_labels.astype(np.int32)
test_data = test_data / np.float32(255)
test_labels = test_labels.astype(np.int32)

使用Tensorflow的feature_column模块定义大小为28\*28的图片，图片像素为28\*28，图片像素值在0-1之间

In [5]:
feature_columns = [tf.feature_column.numeric_column("x", shape=[28, 28])]
feature_columns

[NumericColumn(key='x', shape=(28, 28), default_value=None, dtype=tf.float32, normalizer_fn=None)]

创建logistic回归估计器。使用一个简单的LinearClassifier。后续使用DNNClassifier

In [18]:
LinearClassifier = tf.estimator.LinearClassifier(feature_columns=feature_columns, n_classes=10, model_dir="model\\model/LinearClassifier")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model\\model/LinearClassifier', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


构建一个input_function作为估计器输入

In [19]:
train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(x={"x": train_data}, y=train_labels, batch_size=100, num_epochs=None, shuffle=True)

- **`tf.compat.v1.estimator.inputs.numpy_input_fn`**: 此为TensorFlow 1.x版本中的兼容性函数，旨在确保从1.x向2.x过渡时的代码兼容性。尽管在TensorFlow 2.x中Estimator API依然可用，但现在更推荐使用`tf.data.Dataset`作为数据输入的标准方式。
  
- **`x={"x": train_data}`**: 定义了模型的输入特征字典，其中键`"x"`代表特征列的标识符，`train_data`是对应该特征的实际训练数据集，通常为NumPy数组或其他兼容的数据结构。
  
- **`y=train_labels`**: 提供了目标变量或标签数据，`train_labels`与`train_data`一一对应，包含了每个样本的目标值。
  
- **`batch_size=100`**: 设置了每次迭代时送入模型的样本数量为100。较小的批次有利于内存管理，而较大的批次则可能加速训练过程并提升模型收敛质量。
  
- **`num_epochs=None`**: 指定数据集的遍历次数。设为`None`意味着数据将无限循环直至训练停止条件达成（例如达到预设的迭代步数）。若设定为具体数字，则数据集将被重复使用该指定次数。
  
- **`shuffle=True`**: 启用数据洗牌功能，在每个epoch开始之前对数据进行随机排序。这一步对于避免模型过拟合、提升泛化能力至关重要，因为它确保了模型在训练过程中能接触到数据的不同排列组合。

In [21]:
LinearClassifier.train(input_fn=train_input_fn, steps=10)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into model\model/LinearClassifier\model.ckpt.


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd5 in position 159: invalid continuation byte

- **`LinearClassifier`**: 这是TensorFlow Estimator API中的一个类，用于实现线性模型的分类任务。它适用于那些逻辑回归或多项式逻辑回归能够有效解决问题的场景。

- **`.train(...)`**: 方法用于启动模型的训练过程。Estimator框架内部会根据模型类型（在这里是线性分类器）和提供的参数自动执行训练步骤。

- **`input_fn=train_input_fn`**: `input_fn`是一个函数，负责为模型训练提供数据。在这个例子中，`train_input_fn`是之前定义好的输入函数，它使用`numpy_input_fn`创建，负责从`train_data`和`train_labels`中读取数据，并配置了数据处理的细节，比如批处理大小、是否打乱数据等。

- **`steps=2000`**: 指定了训练的总步数。每一步都涉及到一次或多次（取决于批次大小）模型权重的更新。设置`steps=2000`意味着模型将会基于训练数据迭代2000次，或者直到数据集遍历完成（依据`num_epochs`的设定），以先到者为准。这一步数的选择需要根据实际问题复杂度和模型收敛情况来调整。

In [16]:
val_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(x={"x": test_data}, y=test_labels, num_epochs=1, shuffle=False)

In [17]:
test_result = LinearClassifier.evaluate(input_fn=val_input_fn)
print(test_result)

INFO:tensorflow:Could not find trained model in model_dir: ./model/LinearClassifier, running initialization to evaluate.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2024-06-09T20:27:57
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.55243s
INFO:tensorflow:Finished evaluation at 2024-06-09-20:27:58
INFO:tensorflow:Saving dict for global step 0: accuracy = 0.098, average_loss = 2.3025832, global_step = 0, loss = 2.3025832
{'accuracy': 0.098, 'average_loss': 2.3025832, 'loss': 2.3025832, 'global_step': 0}


这些指标显示模型当前的性能不佳，需要进一步优化。可能的改进措施包括调整模型架构、优化超参数、收集更多或更高质量的数据、调整学习率策略等。

In [22]:
DNNClassifier = tf.estimator.DNNClassifier(feature_columns=feature_columns, hidden_units=[256, 128, 64], n_classes=10, model_dir="./model/DNNClassifier")
DNNClassifier.train(input_fn=train_input_fn, steps=10)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model/DNNClassifier', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INF

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd5 in position 153: invalid continuation byte

In [23]:
test_result = DNNClassifier.evaluate(input_fn=val_input_fn)
print(test_result)

INFO:tensorflow:Could not find trained model in model_dir: ./model/DNNClassifier, running initialization to evaluate.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2024-06-09T20:39:56
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.36801s
INFO:tensorflow:Finished evaluation at 2024-06-09-20:39:57
INFO:tensorflow:Saving dict for global step 0: accuracy = 0.076, average_loss = 2.3452759, global_step = 0, loss = 2.3455408
{'accuracy': 0.076, 'average_loss': 2.3452759, 'loss': 2.3455408, 'global_step': 0}


总结而言，模型的准确率和损失值都有所变动，显示出模型性能的微弱提升，但整体仍然处于较低水平。接下来，考虑加大训练力度，调整模型参数，或引入更有效的训练策略来进一步提升模型性能。同时，确保global_step计数的准确性对于监控模型训练进度也非常重要。